<a href="https://colab.research.google.com/github/NiklasTR/bmi707/blob/master/Assignment2_NiklasRindtorff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 2 - Convolutional Neural Networks (CNNs)

Your Name: Niklas Rindtorff

Name of the Students You Worked With: I asked Ming for advice for the fine-tuning part, but implemented the code myself using the assignment 2 primer.

References You Consulted: Lecture Excercises

# Part 1: Convolutional Neural Networks for Chest X-Ray Images (80 points)
## Data and Preliminaries
In this assignment, we will explore the NIH CXR8 database (https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community). This chest X-ray database is the basis of a series highly-cited papers/preprint papers since 2017. You will examine the images in this dataset and build your own convolutional neural networks to classify the major diagnoses.

## Question 1: Data summary (10 points)
Deep learning is not immune from the 'garbage in, garbage out' principle. Before digging into the data, it is recommended to get a sense of how the data was generated, understand the assumptions of the data, and review the data quality. We will ask you to answer some basic questions on the NIH CXR8 dataset. Please visit the website of the NIH CXR8 database, download the metadata (Data_Entry_2017.csv) and answer the following questions.

####Question 1.1 (5 points)
What is the file format of images in the NIH CXR8 database? What is the standard format for radiology image storage and transmission? How many images are there in the database? How many diagnostic categories are there in the database? What are they? How many images have more than one diagnosis?

__Your Answer:__

The images are stored in the .png format. 
Usually, radiological images are stored in the DICOM format. This datatype is not available in the dataset.
The database comprises 112,120 chest X-rays
There are 14 disease categories. They are: 

* Atelectasis 
* Pneumothorax 
* Pneumonia 
* Pleural Thickening 
* Nodule 
* Mass 
* Infiltration 
* Hernia 
* Emphysema 
* Effusion 
* Edema 
* Consolidation 
* Cardiomegaly




In [1]:
# I load the .csv file and identify the number of images with multiple diagnosis 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# loading packages
!ls gdrive/My\ Drive/'Colab Notebooks'
import pandas as pd

 Assignment1_Niklas_Rindtorff.ipynb
 Assignment2_NiklasRindtorff.ipynb
'Copy of Assignment1_Niklas_Rindtorff.ipynb'
'Copy of deep_learning_regression.ipynb'
'Copy of Hello, Colaboratory'
'Copy of ML_for_images.ipynb'
 Data_Entry_2017.csv
'Deeper semi-continous deep_learning_regression.ipynb'
 deep_learning_regression.ipynb
'Even Deeper semi-continous deep_learning_regression.ipynb'
 Q3Q4
 Q3Q4.zip
 scope.ipynb
'Semicontious deep_learning_regression.ipynb'
 tree_based_regression.ipynb
 Untitled


In [4]:
metadata = pd.read_csv('gdrive/My Drive/Colab Notebooks/Data_Entry_2017.csv')
metadata['Finding Labels'].str.contains('\|').sum()

20796

20796 images contain multiple labels

#### Question 1.2 (5 points)
How many patients are there in total? How many patients contributed more than one image? Which patient ID contributed the most images and how many did he or she contribute?

__Your Answer:__
The dataset contains 30,805 unique patients

In [5]:
metadata.head(10)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
5,00000003_001.png,Hernia,1,3,74,F,PA,2500,2048,0.168,0.168,NaN
6,00000003_002.png,Hernia,2,3,75,F,PA,2048,2500,0.168,0.168,NaN
7,00000003_003.png,Hernia|Infiltration,3,3,76,F,PA,2698,2991,0.143,0.143,NaN
8,00000003_004.png,Hernia,4,3,77,F,PA,2500,2048,0.168,0.168,NaN
9,00000003_005.png,Hernia,5,3,78,F,PA,2686,2991,0.143,0.143,NaN


In [6]:
df_count = (metadata.groupby('Patient ID')
         .agg({'Patient ID' : 'count'})
         .rename(columns={'Patient ID':'count'}))

df_count.query('count > 1').shape

(13302, 1)

13302 patients contributed more than one image. We can see the patient with the most images below:

In [7]:
df_count.sort_values('count', ascending = False).head(1)

,count
Patient ID,
10007,184


## Question 2: Check the images (10 points)
In the following questions, you will be asked to examine the images in the NIH CXR8 dataset. The images for Question 2.1 and 2.2 could be found at https://www.dropbox.com/sh/2h068ge9xv1g27u/AAAXVq8VYXF6HRlHvzvjy-e6a?dl=0. 

Feel free to collaborate with other students or consult any references. For example, this (https://lukeoakdenrayner.wordpress.com/2018/01/24/chexnet-an-in-depth-review/) and many other blog posts provide some contexts.

#### Question 2.1 (2 points)
What is the NIH-labeled diagnosis of image `00001583_014.png`? What did you see in this image? Word limit: 100 words.

__Your Answer:__
The patient has a Pneumothorax in the left lung. We can see a collapsed lung, caused by either a puncture of the thorax, or a spontaneous perforation of a lung bulla. 

#### Question 2.2 (3 points)
What is the NIH-labeled diagnosis of image `00000019_000.png`? What did you see in this image? Word limit: 100 words.

__Your Answer:__
The image has 3 diagnoses: Atelectasis, Effusion and Pleural_Thickening. 

We can see signs of Atelectasis ands effusion in the left lung. A good place to see the effusion between the 3 lobes of the left lung is the white triangle in the center of the left lung. 
I can not identify direct signs of pleural thickening, but it is not a suprising co-diagnosis, given the clnical context. The patient has effusions and, given the sternal cerclage, may had open chest surgery. 

#### Question 2.3 (5 points)
What is "View Position?" How does it affect the resulting chest X-ray images visually? Word limit: 100 words.


__Your Answer:__

Most images were collected PA, that is posteroanterior. Images that are collected this way are collected with the radiation-source coming from the back of the patient. This way, and because the heart is closer to the front of the chest wall, the heart contour is larger in these images. In contrast, AP images, that are collected from sick patients which can not stand in front of a X-ray detector themselves, show smaller heart contours.

## Question 3: Build a custom convolutional neural network (15 points)
For this question, we ask you to build a multi-layer convolutional neural network to classify a subset of cardiomegaly images from normal ones. Please download the training set and the validation set here (https://www.dropbox.com/sh/ojiw79q8786ua4x/AAAtaJVKEdv91Zybpi-fAfMsa?dl=0). Please DO NOT use any other image from NIH CXR8 or from other databases for this question. Feel free to use keras or any other high-level deep learning packages to classify the images.

Design a convolutional neural network with at least two convolution layers, at least one max-pooling layer, and at least one dropout layer. Although you should explore various combinations of hyperparameters, we will grade this question based on the accuracy of the implementation, not the performance of the network.

What is your design? What binary loss/accuracy did you get in the training and validation set? Please include your code in the assignment submission.

Your neural network should have an AUC >= 0.55 when evaluated by the validation data set.

### Pre-process data

In [8]:
# I list my image data
! ls gdrive/'My Drive'/'Colab Notebooks'/Q3Q4/converted_to_RGB

train  train.tar  val  val.tar


In [9]:
# and I list my label information
! ls gdrive/'My Drive'/'Colab Notebooks'/Q3Q4
y_train = pd.read_csv('gdrive/My Drive/Colab Notebooks/Q3Q4/train.csv', header=None, names = ['image', 'label'])
y_test = pd.read_csv('gdrive/My Drive/Colab Notebooks/Q3Q4/val.csv', header=None, names = ['image', 'label'])


converted_to_RGB  train.csv  val.csv


In [0]:
#! mkdir gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/train/Cardiomegaly
#! mkdir gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/train/No\ Finding
#! mkdir gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/val/Cardiomegaly
#! mkdir gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/val/No\ Finding



In [0]:
import shutil


y_train['full_path'] = 'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/train/' + y_train['image']
y_test['full_path'] = 'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/val/' + y_test['image']

y_train['destination_path'] = 'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/train/' + y_train['label'] + '/' + y_train['image']
y_test['destination_path'] = 'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/val/' + y_test['label'] + '/' + y_test['image']

# only run once
#y_train.apply(lambda row: shutil.move(row['full_path'], row['destination_path']), axis=1)
#y_test.apply(lambda row: shutil.move(row['full_path'], row['destination_path']), axis=1)

In [11]:
! ls gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/val/Cardiomegaly

00000001_000.png  00005035_004.png  00012463_000.png  00020754_000.png
00000013_045.png  00005064_013.png  00012523_000.png  00020839_002.png
00000032_000.png  00005066_065.png  00012707_000.png  00020979_001.png
00000032_053.png  00005270_007.png  00012798_000.png  00021032_000.png
00000045_000.png  00005425_000.png  00012845_001.png  00021058_000.png
00000155_000.png  00005532_007.png  00012987_002.png  00021186_001.png
00000176_001.png  00005641_001.png  00012987_012.png  00021326_000.png
00000211_000.png  00005906_000.png  00013409_001.png  00021373_010.png
00000271_004.png  00006068_002.png  00013522_000.png  00021790_000.png
00000287_000.png  00006129_000.png  00013876_001.png  00021818_023.png
00000330_000.png  00006341_000.png  00013876_006.png  00021895_007.png
00000338_000.png  00006411_017.png  00013876_007.png  00021958_000.png
00000374_000.png  00006481_015.png  00013992_017.png  00022147_005.png
00000547_005.png  00006537_000.png  00014022_084.png  00022945_000.png
000008

In [12]:
# I create a data-generator for my custom network
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        #samplewise_center=True, 
        #samplewise_std_normalization=True,
        horizontal_flip=False) # The heart is always on the left side

test_datagen = ImageDataGenerator(rescale=1./255)#,
                                  #samplewise_center=True, 
                                  #samplewise_std_normalization=True)

# define the batch size, if there is sufficient GPU memory, you can increase the batch size
batch_size = 16

# I define the training data generator
train_generator = train_datagen.flow_from_directory(
        'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/train',  # the directory for the training data
        target_size=(150, 150),  # resize the input images to accommodate the model
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/val',  # the directory for the training data
        target_size=(150, 150),  # resize the input images to accommodate the model
        batch_size=batch_size,
        class_mode='binary')

Found 1750 images belonging to 2 classes.
Found 437 images belonging to 2 classes.


### Train custom network

In [0]:
# loading packages
################################################################################
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten, Dropout, MaxPooling2D

# setting variables
batch_size = 16



In [0]:
#Design a convolutional neural network with at least two convolution layers, at least one max-pooling layer, and at least one dropout layer
model = Sequential()

# adding layers 
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
model.fit_generator(train_generator,
                    epochs=10, 
                    steps_per_epoch=1750 // batch_size,
                    validation_data=test_generator,
                    validation_steps=437 // batch_size)


In [22]:
import numpy as np
from sklearn.metrics import roc_auc_score

# I measure the networks AUC on the validation dataset. I called this dataset test dataset, which it isn't. 
# I create a dedicated generator without randomization and a batchsize of 1
generator = test_datagen.flow_from_directory(
        'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/val',
        target_size=(150, 150),
        batch_size=1,
        class_mode=None,
        shuffle=False)


Found 437 images belonging to 2 classes.


In [0]:
# I calculate the class probabilities
probabilities = model.predict_generator(generator, steps = 437)


# Suprisingly, I do not need to binarize the class labels first
roc_auc_score(y_test['label'], probabilities)

__Your Answer:__

My final network consists of 3 convolutional layers that are interspersed with Max-pooling layers. The data is flattened and then processed in two fully connected layers. One of the fully connected layers is dropout regularized. 

The accuracy and loss during training reached up to 0.59 and 0.69, repsectively. The metrics on the validation data reached up to 0.65 and 0.69, respectively. 

The AUROC on the validation data is larger than 0.55.

## Question 4: Transfer learning: Using the VGGNet (16 layers) architecture (20 points)
For this question, we ask you to employ VGGNet, a convolutional neural network built for ImageNet, to classify the same subset of cardiomegaly images from normal ones (https://www.dropbox.com/sh/ojiw79q8786ua4x/AAAtaJVKEdv91Zybpi-fAfMsa?dl=0). We encourage you to take a look at the documentation for keras.applications (https://keras.io/applications/) and reuse their modules. Please DO NOT use any other images from NIH CXR8 or from other databases for this question. Although you should explore various combinations of hyperparameters, we will grade this question based on the accuracy of the implementation, not the performance of the network.

Your neural network should have an AUC >= 0.7 when evaluated by the validation data set.

In [35]:
# First, I generate my training data in a new way, that is independent of the image generator. I struggled with fine-tuning the network using generators, and resorted to manually importing the information
!tar -xf gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/train.tar
!tar -xf gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/val.tar

!ls gdrive/My\ Drive/Colab\ Notebooks/Q3Q4/converted_to_RGB/

train_cat  train.tar  val_cat  val.tar


In [0]:
# I import packages
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from PIL import Image
from skimage import color
from skimage import io
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelBinarizer

##################
# TRAIN data
# read in the labels
train_labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/Q3Q4/train.csv',  header=None, index_col=0)
# Any results you write to the current directory are saved as output.
# I resize the images to make them smaller
image_size = (224,224)

# read in the training images
train_images = []
train_dir = 'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/train/'

train_files = train_labels.index.values.tolist()

for f in train_files:
  img = Image.open(train_dir + f)
  img = img.resize(image_size)
  img_arr = np.array(img)
  train_images.append(img_arr)

train_X = np.array(train_images)
# reorder the labels so that they line up with the order of the image files
train_labels = train_labels.reindex(train_files)
print(train_X.shape)


# I scale the images across the full dataset
train_X = train_X/255
train_X = (train_X - train_X.mean(axis=0))/train_X.std(axis=0)

train_X = color.gray2rgb(train_X)
train_X.shape

# I create binary labels for my y variable
label_transformer = LabelBinarizer()
train_y = label_transformer.fit_transform(train_labels)


In [4]:
#####################
# VALIDATION data
# read in the labels
val_labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/Q3Q4/val.csv',  header=None, index_col=0)

# read in the training images
val_images = []
val_dir = 'gdrive/My Drive/Colab Notebooks/Q3Q4/converted_to_RGB/val/'

val_files = val_labels.index.values.tolist()

for f in val_files:
  img = Image.open(val_dir + f)
  img = img.resize(image_size)
  img_arr = np.array(img)
  val_images.append(img_arr)

val_X = np.array(val_images)
# reorder the labels so that they line up with the order of the image files
val_labels = val_labels.reindex(val_files)
print(val_X.shape)


# I scale the images across the full dataset
val_X = val_X/255
val_X = (val_X - val_X.mean(axis=0))/val_X.std(axis=0)

val_X = color.gray2rgb(val_X)
val_X.shape

# I create binary labels for my y variable
label_transformer = LabelBinarizer()
val_y = label_transformer.fit_transform(val_labels)

(437, 224, 224)


### Question 4.1 (10 points)
What is your best validation accuracy of fine-tuning a 16-layer VGGNet WITHOUT ImageNet weights? In your model with the lowest validation loss, what are the hyperparameters? What is the validation loss? What is the training loss/accuracy? Please include your code in the assignment submission.

In [10]:
## I take inspiration from the in-class assignment for this code
################################################################################
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam,SGD,RMSprop
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten, Dropout, MaxPooling2D


base_model = VGG16(weights=None, include_top=True) #different from the in-class excercise, I keep the classification layer and just add another set of layers on top.
# get the output of the base model
x = base_model.output
# add a 2D global average pooling layer
x = Dense(1024, activation='relu')(x)
# I add a regularizing layer
x = Dropout(0.5)(x)
# add a layer for binary classification. As above, I use a sigmoid activation function
predictions = Dense(1, activation='sigmoid')(x)

model_vvg_random = Model(inputs=base_model.input, outputs=predictions)
    
# I compile the model and start training using the previously defined generators
# I keep the optimizer and loss function constant for consistency
model_vvg_random.compile(optimizer=RMSprop(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train model with same schedule, I did not freeze any layers, as the weights are all random
model_vvg_random.fit(train_X,train_y, validation_data=[val_X,val_y], epochs=35,batch_size=32)



Train on 1750 samples, validate on 437 samples
Epoch 1/35
1750/1750 [==============================] - 35s 20ms/step - loss: 0.6932 - acc: 0.4960 - val_loss: 0.6931 - val_acc: 0.4989
Epoch 2/35
1750/1750 [==============================] - 32s 18ms/step - loss: 0.6932 - acc: 0.4886 - val_loss: 0.6931 - val_acc: 0.4989
Epoch 3/35
1750/1750 [==============================] - 31s 18ms/step - loss: 0.6906 - acc: 0.5680 - val_loss: 0.6890 - val_acc: 0.6110
Epoch 4/35
1750/1750 [==============================] - 32s 18ms/step - loss: 0.6862 - acc: 0.6046 - val_loss: 0.6888 - val_acc: 0.5789
Epoch 5/35
1750/1750 [==============================] - 32s 18ms/step - loss: 0.6833 - acc: 0.6246 - val_loss: 0.6852 - val_acc: 0.6133
Epoch 6/35
1750/1750 [==============================] - 32s 18ms/step - loss: 0.6828 - acc: 0.6097 - val_loss: 0.6832 - val_acc: 0.6201
Epoch 7/35
1750/1750 [==============================] - 32s 18ms/step - loss: 0.6807 - acc: 0.6320 - val_loss: 0.6846 - val_acc: 0.6064
E

In [11]:
pred_y = model_vvg_random.predict(val_X)
print("The AUROC on the validation data is ", roc_auc_score(val_y, pred_y))


The AUROC on the validation data is  0.6324724561183025


__Your Answer:__

For the VGGNet architecture with random initializations, the network does learn to differentiate between the two classes when trained with a different set of hyperparameters. However, the model performance after 35 epochs is still below AURC = 0.7. A possible reason for this could be that the VGGNet achitecture has too many parameters, that can not converge to a robust configuration with the limited available training data and training iterations.

### Question 4.2 (10 points)
What is your best validation accuracy of fine-tuning a 16-layer VGGNet WITH ImageNet weights? In your model with the lowest validation loss, what are the hyperparameters? What is the validation loss? What is the training loss/accuracy? Please include your code in the assignment submission.

In [9]:
## I keep everyhting the same as above, but load the imagenet features
################################################################################
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=True)
# get the output of the base model
x = base_model.output
# get the output of the base model
x = base_model.output
# add a 2D global average pooling layer
x = Dense(1024, activation='relu')(x)
# I add a regularizing layer
x = Dropout(0.5)(x)
# add a layer for binary classification. As above, I use a sigmoid activation function
predictions = Dense(1, activation='sigmoid')(x)

model_vvg_imagenet = Model(inputs=base_model.input, outputs=predictions)


    
# I compile the model and start training using the previously defined generators
# I keep the optimizer and loss function constant for consistency
model_vvg_imagenet.compile(optimizer=RMSprop(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train model with same schedule, I did not freeze any layers, as the weights are all random
model_vvg_random.fit(train_X,train_y, validation_data=[val_X,val_y], epochs=35,batch_size=32)

pred_y_inception = model_vvg_random.predict(val_X)

print("The AUROC on the validation data is ", roc_auc_score(val_y, pred_y_inception))


Train on 1750 samples, validate on 437 samples
Epoch 1/35
1750/1750 [==============================] - 32s 18ms/step - loss: 0.6509 - acc: 0.6811 - val_loss: 0.6603 - val_acc: 0.6476
Epoch 2/35
1750/1750 [==============================] - 32s 18ms/step - loss: 0.6504 - acc: 0.6766 - val_loss: 0.6628 - val_acc: 0.6339
Epoch 3/35
1750/1750 [==============================] - 31s 18ms/step - loss: 0.6408 - acc: 0.7057 - val_loss: 0.6581 - val_acc: 0.6476
Epoch 4/35
1750/1750 [==============================] - 31s 18ms/step - loss: 0.6426 - acc: 0.6931 - val_loss: 0.6613 - val_acc: 0.6339
Epoch 5/35
1750/1750 [==============================] - 31s 18ms/step - loss: 0.6350 - acc: 0.7120 - val_loss: 0.6598 - val_acc: 0.6362
Epoch 6/35
1750/1750 [==============================] - 31s 18ms/step - loss: 0.6302 - acc: 0.7206 - val_loss: 0.6568 - val_acc: 0.6430
Epoch 7/35
1750/1750 [==============================] - 31s 18ms/step - loss: 0.6409 - acc: 0.6840 - val_loss: 0.6728 - val_acc: 0.5995
E

__Your Answer:__

When using the pre-trained parameters from the ImageNet-optimized VGGNet and the same hyperparameter configuration we can observe an AUROC of > 0.7. With the limited number of training data and training epochs, the model was still able to reach a higher performance, because the model parameters were already pre-optimized. 

## Question 5: Multi-class classification and the BMI707 Kaggle contest (20 points)
In this question, we will build multi-class classifiers to distinguish different types of lung diseases using the NIHCXR8 data.
Please download the training set and the validation set from the BMI707 Kaggle contest website (https://www.kaggle.com/c/2019bmi707-assignment-2-q5/data ). Please note that this dataset is different from the one we used in Question 3 and 4. Please DO NOT use any additional dataset (including those from NIH CXR8) to train or augment your models. Feel free to use any (ImageNet or any custom) architecture to classify all available classes. In your model with the lowest validation loss, what are the hyperparameters? What is the validation loss/accuracy? What is the training loss/accuracy? Please participate in the BMI707 internal Kaggle contest (https://www.kaggle.com/c/2019bmi707-assignment-2-q5) and compare your results with others there. An ensemble of models is allowed. The top 5 submissions with the highest accuracy on the private test set (testPrivate.tar) will receive bonus points. Please include your code in the assignment submission.

### The below code was submitted on kaggle and returned the second highest score on the scoreboard (as of May 1st, 2 PM). The code does not run in the Google Colab notebook.

In [0]:
# navigate the repo
import os
print(os.listdir("../input"))

In [0]:
# I copy this chunk from the primer to load the data into the kernel
from PIL import Image
import numpy as np
import pandas as pd
import os

# read in the labels
train_labels = pd.read_csv('../input/train.csv',  header=None, index_col=0)

# I resize the images to make them smaller
image_size = (224,224)

# read in the training images
train_images = []
train_dir = '../input/train/train/'
train_files = os.listdir(train_dir)
for f in train_files:
  img = Image.open(train_dir + f)
  img = img.resize(image_size)
  img_arr = np.array(img)
  train_images.append(img_arr)

train_X = np.array(train_images)

In [0]:

# read in the test images
test_p_images = []
test_pri_dir = '../input/testprivate/test/'
test_pub_dir = '../input/testpublic/val/'
test_pri_files = os.listdir(test_pri_dir)
test_pub_files = os.listdir(test_pub_dir)

# load data
for f in test_pri_files:
  img = Image.open(test_pri_dir + f)
  img = img.resize(image_size)
  img_arr = np.array(img)
  test_p_images.append(img_arr)
for f in test_pub_files:
  img = Image.open(test_pub_dir + f)
  img = img.resize(image_size)
  img_arr = np.array(img)
  test_p_images.append(img_arr)

test_p_X = np.array(test_p_images)

# create a common index 
test_p_files = test_pri_files + test_pub_files

# reorder the labels so that they line up with the order of the image files
train_labels = train_labels.reindex(train_files)

In [0]:
# I print an example image
import matplotlib.pyplot as plt
plt.imshow(train_images[0], cmap='gray')
plt.show()

In [0]:
# I binarize my data
from sklearn.preprocessing import LabelBinarizer
label_transformer = LabelBinarizer()
train_y = label_transformer.fit_transform(train_labels)

# I display the first row
train_y[1,:]

In [0]:
# Finally, I add a dimension for the channel
train_X = np.repeat(train_X[:, :, :, np.newaxis], 3, axis=3)
test_p_X = np.repeat(test_p_X[:, :, :, np.newaxis], 3, axis=3)

In [0]:
# I also scale my images. This is their properties before pre-processing
train_X.max()
train_X.min()
train_X.mean()

train_X.shape

In [0]:
test_p_X.shape

In [0]:
# I define my data generators for training and testing
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        samplewise_center=True, 
        samplewise_std_normalization=True,
        horizontal_flip=False) # The heart is always on the left side

test_datagen = ImageDataGenerator(rescale=1./255,
                                  samplewise_center=True, 
                                  samplewise_std_normalization=True)



In [0]:
# compute mean and sd in case of featurewise scaling. In this case I scaled every image independently
#train_datagen.fit(train_X)
#test_datagen.fit(test_p_X)

In [0]:
# define model
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten, Dropout, MaxPooling2D

base_model = NASNetMobile(weights='imagenet', include_top=False)
# get the output of the base model
x = base_model.output
# I add a flatten operation 
x = GlobalAveragePooling2D()(x)
# add a layer for binary classification. As above, I use a sigmoid activation function
predictions = Dense(15, activation='sigmoid')(x)


model_nasnet_imagenet = Model(inputs=base_model.input, outputs=predictions)

# I compile the model and start training using the previously defined generators
# I keep the optimizer and loss function constant for consistency
model_nasnet_imagenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# setting variables
batch_size = 16

# I train the network
model_nasnet_imagenet.fit_generator(train_datagen.flow(train_X, train_y, batch_size=batch_size),
                    epochs=10, 
                    steps_per_epoch=5483 // batch_size)

In [0]:
prediction = model_nasnet_imagenet.predict_generator(test_datagen.flow(test_p_X, batch_size = 1), steps = 3654)

df = pd.DataFrame(test_p_files, columns=['Id'])
df['Category'] = np.argmax(prediction, axis=1)
#df.head()

df.shape

df.to_csv('submission.csv', index=False)

## Question 6: Limitations of the NIH CXR8 dataset and the road ahead (5 points)

Please list three limitations of models trained from this dataset. Word limit: 150 words.

__Your Answer:__


1.   Models trained on this dataset are optimized for the interpretation of X-rays that were collected in only one US hospital PACS system. Thus, the generalizability of models trained on this dataset might be limited.
1.   The labels in the dataset were generated using natural language processing and were not evalauted by a set of independent radiologists. Thus the labels could be erroneuos.
1.   Most images contain procedure-related artefacts, such as a label with the mode of acquisition or an indicator of X-ray orientation. This can lead to information diffusion, in which non-morphological data is linked to a diagnosis. 



# Part 2: Recurrent Neural Networks (20 points)

During the in-class exercise of Lecture 4, we implemented a simple recurrent neural network for classifying newswires from Reuters. In this question, you will implement a recurrent neural network model using long short-term memory (LSTM) or gated recurrent unit (GRU), and compare its performance with that of the simple recurrent neural network in the in-class exercise using the same dataset. 

## Question 7: Build a recurrent neural network with LSTM or GRU for text classification (15 points)



In [1]:
# I have to downgrade numpy to load the dataset 
!pip install numpy==1.16.2
import numpy as np
print(np.__version__)

    100% |████████████████████████████████| 17.3MB 2.9MB/s 
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Successfully uninstalled numpy-1.16.3


1.16.3


In [0]:
# For fairness, I download the same dataset 

# load the required packages
import keras
from keras.layers import SimpleRNN, Dense, Embedding, GRU, Dropout
from keras.models import Sequential
from keras.datasets import reuters
from keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
maxlen = 500          # cut texts after this number of words
batch_size = 64

# load data from keras.datasets
(x_train_r, y_train_r), (x_test_r, y_test_r) = reuters.load_data(num_words=max_features)

In [5]:
## Your code goes here
################################################################################
# I transform my data by padding, trimming and one-hot encoding
from keras.utils import np_utils

y_train_r = np_utils.to_categorical(y_train_r, 46)
y_test_r = np_utils.to_categorical(y_test_r, 46)

x_train_r = sequence.pad_sequences(x_train_r, maxlen=maxlen)
x_test_r = sequence.pad_sequences(x_test_r, maxlen=maxlen)
print('After padding:')
print('The shape of x_train:', x_train_r.shape)
print('The shape of x_test:', x_test_r.shape)
print('The shape of y_train:', y_train_r.shape)
print('The shape of y_test:', y_test_r.shape)

After padding:
The shape of x_train: (8982, 500)
The shape of x_test: (2246, 500)
The shape of y_train: (8982, 46)
The shape of y_test: (2246, 46)


In [6]:
n_epochs=3

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(GRU(64, return_sequences=True))
model.add(GRU(64, return_sequences=True))
model.add(GRU(64, return_sequences=True))
model.add(GRU(64))
model.add(Dropout(0.2))
model.add(Dense(46, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train_r, y_train_r,
                    epochs=n_epochs,
                    batch_size=128,
                    validation_split=0.3)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 6287 samples, validate on 2695 samples
Epoch 1/3
6287/6287 [==============================] - 136s 22ms/step - loss: 2.7656 - acc: 0.3356 - val_loss: 2.1464 - val_acc: 0.3570
Epoch 2/3
6287/6287 [==============================] - 130s 21ms/step - loss: 2.0128 - acc: 0.4333 - val_loss: 1.8777 - val_acc: 0.4950
Epoch 3/3
6287/6287 [==============================] - 132s 21ms/step - loss: 1.7565 - acc: 0.5179 - val_loss: 1.7867 - val_acc: 0.5369


In [7]:
# I collect the performance values similar to the in-class excercise
score = model.evaluate(x_test_r, y_test_r, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2246/2246 [==============================] - 31s 14ms/step
Test loss: 1.817584462390876
Test accuracy: 0.5298308103294747


## Question 8: How does the performance of your model compare with that of the simple recurrent neural network model in the in-class exercise? Why? (5 points)

Word limit: 100 words.

__Your Answer:__

Compared to the simple RNN architecture in the in-class excercise, the model had a higher accuracy (0.42 vs. 0.53) on the test dataset and the same evaluation method. A possible reason for this could be that the cell state can be forgotten in GRU-, compared to SimpleRNN cells. Still, even the GRU-powered network performed worse than the CNN from our in-class excercise. 